In [ ]:
# Install

!git clone https://github.com/sebastien-ethereum/BenchMARL
%cd /content/BenchMARL
!pip install -e .

In [ ]:
# Refresh git repo

%cd /content/BenchMARL
!git stash
!git pull

In [ ]:
# Environment sanity check

from torchrl.envs.utils import check_env_specs
from benchmarl.environments.auction1.env import Auction1Env

env = Auction1Env(num_envs=100, seed=0, device="cpu")
check_env_specs(env)

In [ ]:
# Experiment

from benchmarl.environments import Auction1Task
from benchmarl.algorithms import IppoConfig
from benchmarl.models.mlp import MlpConfig
from benchmarl.experiment import ExperimentConfig, Experiment
from benchmarl.callbacks.npc_eval_callback import NpcEvalCallback

seed = 1

task              = Auction1Task.AUCTION1.get_from_yaml()
algorithm_config  = IppoConfig.get_from_yaml()
model_config      = MlpConfig.get_from_yaml()
experiment_config = ExperimentConfig.get_from_yaml()

task.update_config({
    "num_agents": 10,
    "npcs": {},
})

model_config.num_cells = [8]

experiment_config.on_policy_n_envs_per_worker = 6000
experiment_config.off_policy_n_envs_per_worker = 6000
experiment_config.evaluation_episodes = 6000
experiment_config.evaluation_interval = 60000

experiment_config.loggers = ["wandb"]

callbacks = [NpcEvalCallback()]

experiment = Experiment(
    task             = task,
    algorithm_config = algorithm_config,
    model_config     = model_config,
    seed             = seed,
    config           = experiment_config,
    callbacks        = callbacks,
)

experiment.run()